# Minimal: Scan scaler *vs* motor

*APS Training for Bluesky Data Acquisition*.

**Objective**

Connect an EPICS motor PV, an EPICS scaler PV, and scan using the bluesky *RunEngine* with the *BestEffortCallback* to visualize the acquired data.

EPICS PV | Python object name | description
:--- | :--- | :---
`gp:m1` | `xpos` | motor PV (simulates a stepper motor)
`gp:scaler1` | `vsc16` | scaler PV (simulates a Joerger VSC16 scaler)

In [ ]:
# first, load the environment variable definitions
%load_ext dotenv
%dotenv env.txt

## Connect the motor

Import the `ophyd.EpicsMotor` (class) definition.

In [ ]:
from ophyd import EpicsMotor

We'll need the prefix of the custom EPICS IOC for this user account.  It is defined in the environment variable `GP_IOC_PREFIX`.  (If that environment variable is not defined, as on the developer's workstation, default to `gp:`.)

In [ ]:
import os
GP = os.environ.get("GP_IOC_PREFIX", "gp:")
print(f"{GP = }")

Then, define the `xpos` object.  The EPICS PV is the first argument.  The `name="xpos"` is
required.  Make the value of this keyword argument match the name of the Python object being assigned.  The `labels=("motor",)` keyword argument enables certain features for the user interface that will be described later.

In [ ]:
xpos = EpicsMotor(f"{GP}m1", name="xpos", labels=("motor",))

Wait for the EPICS PV to connect with Python.

In [ ]:
xpos.wait_for_connection()

Show the position of the `xpos` motor now.

In [ ]:
xpos.position

Try to move `xpos` using the IPython magic command: `%mov`.

In [ ]:
%mov xpos 1

Note this command fails since the IPython magic commands are not loaded automatically.  As an alternative, use the motor's `.move(SETPOINT)` method to move the motor to the new *SETPOINT* value.  Here, move `xpos` to `1`.

In [ ]:
xpos.move(1)

The `.move()` method returns a status object.  The status object may be used to report about the move or to wait for the motor to complete its move.

Next, load the IPython *Magic* commands provided by `bluesky`.  These are helpers to make the command line use easier.  Use them at your choice.

In [ ]:
from bluesky.magics import BlueskyMagics
get_ipython().register_magics(BlueskyMagics)

Repeat the move of `xpos` by sending it to `0` using the magic command.

In [ ]:
%mov xpos 0

## Connect the scaler
As before, import the ophyd class:

In [ ]:
from ophyd.scaler import ScalerCH

Then, connect with the EPICS PV.  This is similar to how the motor was connected (above).  Make the `name=` keyword match with the Python object name being created.   The `labels=("scalers", "detectors")` keyword argument enables certain features for the user interface that will be described later.

In [ ]:
vsc16 = ScalerCH(f"{GP}scaler1", name="vsc16", labels=["scalers", "detectors"])
vsc16.wait_for_connection()

Configure the Python object to ignore the channels with no name defined (in the `.NMnn` field of the scaler PV).

In this example control screen for our scaler, only a few of the channels are named:

![`scaler` GUI](resources/scaler.png "`scaler1` GUI")

In [ ]:
vsc16.select_channels()

Show the value for each of the named channels in **our** scaler.  This will also include the most recent count time.

In [ ]:
vsc16.read()

If *no* channels are shown, such as:

```python
OrderedDict([('vsc16_time', {'value': 0.0, 'timestamp': 631152000.0})])
```

then we want to name some channels here for our simulator.  We can do this directly from Python (although you would not usually want to do this at a real, operating instrument, since the channels are named as a result of hardware connections with real detectors).  We'll take channels 11 & 12 and call them `roi1` & `roi2`, respectively.

In [ ]:
vsc16.channels.chan01.chname.put("timebase")
vsc16.channels.chan11.chname.put("roi1")
vsc16.channels.chan12.chname.put("roi2")

Then select the named channels and read again:

In [ ]:
vsc16.select_channels()
vsc16.read()

### Set the scaler's counting time

The `ScalerCH` class defines the count time as the `preset_time` attribute.  Show it's value:

In [ ]:
vsc16.preset_time.get()

Set the counting time to 0.5 s using the `%mov` magic command (same command that moves a motor).

In [ ]:
%mov vsc16.preset_time 0.5

Use the IPython `%ct` magic command to count each object with the `detectors` label keyword.

In [ ]:
%ct

### `labels` and the `%wa` magic command

As noted above, the `labels=LIST` keyword argument used when the motor and scaler objects were create enable certain features.  For example, the `%ct` magic command will count all the `detectors`.

Actually, `detectors` is the default argument for the `%ct` magic.  If an arument is supplied, it is the name of a label to be matched.  This is why the additional label of `scalers` was included.  Thus, we could count *only* `scalers` with the command `%ct scalers`.

Use the `labels` keyword liberally to group similar objects.

The `%wa` magic shows relevant information for all labeled objects (or, for the named label if supplied).  For example:

In [ ]:
%wa

In [ ]:
%wa motor

## Prepare to scan

Before we can run our first bluesky scan, we have to import various software tools.

First is the bluesky `RunEngine()`, which will manage the various activities for the scan (move motor, wait, trigger scaler, wait, collect channel data, publish to data subscribers, ...).

Create a `RunEngine()` object.  The argument here is a dictionary (empty in this training session).  (For routine operations at an instrument, the dictionary is filled with information saved from the previous session.)

In [ ]:
import bluesky
RE = bluesky.RunEngine({})

To save the acquired data, we connect with a MongoDB database using a preconfigured datafile that describes our databroker catalog (it's in the file named in the next cell).  Using the IPython `!` technique to issue a Linux command from an IPython session, we `cat` (concatenate) the contents of that file to the output here:

In [ ]:
!cat ~/.local/share/intake/training.yml

To connect, we need the *training* catalog.  This name is provided by the line indented after the `sources:` line in the above `.yml` file.  (The name of the `.yml` file does not matter.  The `databroker.catalog` software will look through all `.yml` files in this directory for the`training` configuration.)

We'll use the reference to our databroker catalog frequently, so we give it a short name.

In [ ]:
import databroker
cat = databroker.catalog["training"]

How many (bluesky data collection) *runs* are recorded in this catalog?  Get its *length*:

In [ ]:
len(cat)

Configure `RE` to publish the run data to our `db` object.  We must use the `.v1` software interface for legacy reasons.

In [ ]:
RE.subscribe(cat.v1.insert)

A progress bar can be helpful to show that long operations are actually progressing.  These steps load a progress bar and configure `RE`.

In [ ]:
from bluesky.utils import ProgressBarManager
pbar_manager = ProgressBarManager()
RE.waiting_hook = pbar_manager

The `BestEffortCallback` provides easy visualization of data (tables, plots, peaks statistics) as it is acquired by the `RE`.  Subscribe it to the `RE` so it receives data during a `RE()` run.

In [ ]:
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
RE.subscribe(bec)
peaks = bec.peaks

Supplemental *baseline* data is recorded before and after each run.  Additionally, EPICS Channel Access monitors can update PVs asynchronous to the primary data acquisition.  These *monitor*s can be saved as additional data streams in a run.  Prepare to use this feature.

In [ ]:
from bluesky import SupplementalData
sd = SupplementalData()
RE.preprocessors.append(sd)

Add the name of this notebook as metadata to every run.  This is done by adding to the RunEngine's metadata dictionary (`RE.md`), content that will be added to the *start* document of every run.  The metadata is useful documentation about a run and can be used for several purposes, such as to record a general condition (such as the name of this notebook) or to identify these runs from a database search.

In [ ]:
RE.md["notebook"] = "basic-motor-scaler-scan"

## First scan

The standard plans provided in `bluesky.plans` are sufficient for many needs, so import them and, in the same command, give the package a short name (`bp`) since it is used frequently.

In [ ]:
from bluesky import plans as bp

The `scan()` plan is flexible and will be used here to scan scaler *vs.* motor.  The first argument is the *list* of detectors to be recorded.  Here, we give the scaler object `vsc16`.  The next argument is the positioner object, then start and end positions, finally, the number of points to be collected.

Observe that we do not run the scan directly, but rather give the scan to the `RE()` object.  The `RE()` object will run the scan, performing each of the actions defined by the scan, but also handle the additional tasks of managing the data acquisition process, publishing data to all subscribers (here: databroker and BestEffortCallback) and checking for updates from EPICS and checking if the run must be interrupted either by user request or some other observation.  (We have not configured any of those other observations in this simple example.)

In [ ]:
RE(bp.scan([vsc16], xpos, -1, 1, 7))

The scan ran, data was collected and printed at each step of the scan.  Finally, plots were made of the scaler channel *vs* motor position for each active channel.

There were some problems.  First is that an error was reported after the scan (`... callbacks/fitting.py:165: RuntimeWarning: invalid value encountered in double_scalars `).  This error is because the scalers showed no peak during the scan.  The scaler is a simulator with no real data.  We'll ignore that error here.

Another problem is that the scaler counts for 0.1s longer than we have configured.  This is a problem with the underlying EPICS support for a simulated (a.k.a. *soft channel*) scaler.  We'll ignore that error here.

Next problem is that the progress bar is a nuisance in this notebook so we'll remove it.

In [ ]:
RE.waiting_hook = None

Then, repeat the same scan.

In [ ]:
RE(bp.scan([vsc16], xpos, -1, 1, 7))

## Scan with a *different* counting time : staging

Suppose that we wish to use a different counting time, we could change the `vsc16.preset_time` value before running the scan.  Another way is to use the `ophyd` concept of [*stage* & *unstage*](https://blueskyproject.io/ophyd/device-overview.html?highlight=staging#stage-and-unstage).  

*Staging* is the action of preparing an ophyd device for operation, then resetting it afterwards to previous values.  For our scaler, we could *stage* a different counting time that would be used during the run, then removed after the run is complete.  `stage()` and `unstage()` are controlled by an OrderedDictionary where the keys are the attributes of the Python object and the values are used during the run.  The `RE()` takes care of calling `stage()` and `unstage()` during the scan.

Here we show staging of a 2.0s `preset_time` for the run.  Also shown are the `preset_time` value before and after the run.

In [ ]:
print(f"{vsc16.preset_time.get() = }")

In [ ]:
vsc16.stage_sigs["preset_time"] = 2.0

Repeat the same scan.

In [ ]:
RE(bp.scan([vsc16], xpos, -1, 1, 7))

In [ ]:
print(f"{vsc16.preset_time.get() = }")

## Custom plan with configurable count time

It is common to want to set the count time at the time the scan is started.  For this feature, a custom scan plan is needed, where we will repeat the steps just shown.  This plan will use similar arguments as the `bp.scan()` used above, but add an optional keyword argument `ct` for the count time with a default of 1.0 second.  For housekeeping, we'll remove the staging configuration from the scaler after the scan.

A bluesky *plan* is a Python generator.  The `bp.scan()` call is the part that makes this a generator, deferring execution of the actual scan until the `RE()` calls for it.

In [ ]:
def tscan(scaler, pos, pStart, pEnd, nPts, ct=1):
    scaler.stage_sigs["preset_time"] = ct
    print(f"{scaler.preset_time.get() = }")

    yield from bp.scan([scaler], pos, pStart, pEnd, nPts)

    print(f"{scaler.preset_time.get() = }")
    del scaler.stage_sigs["preset_time"]

Run the custom plan.

In [ ]:
RE(tscan(vsc16, xpos, -1, 1, 7, 0.1))

## Challenges

Try these additional modifications or activities.

1. Write a custom plan that accepts user-provided metadata.